In [2]:
import pandas as pd
import plotly.graph_objects as go
import requests
from datetime import datetime
import numpy as np
from time import sleep

    
def show_price(df):
    if df is None or df.empty:
        return
    fig = go.Figure(data=[go.Candlestick(x=df['opentime'],
                    open=df['open'], high=df['high'], low=df['low'], close=df['close'])])
    fig.show()

# use binance as data source
def get_symbols():
    r = requests.get("https://api.binance.com/api/v3/exchangeInfo")
    if not r:
        print("data empty")
        return None
    d = r.json()['symbols']
    return [item['symbol'].replace("USDT","") for item in d if 'USDT' in item['symbol']]
def get_price(symbol, period):
    url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval={period}"
    r = requests.get(url)
    if not r:
        print("data empty")
        return None
    d = r.json()
    df = pd.DataFrame(d, columns=['opentime','open','high','low',
                              'close','volume','closetime',
                              'vcoin','count','buyvolume','buycoin','ignore'])
    df['opentime'] = pd.to_datetime(df['opentime'], unit='ms')
    df['close'] = df['close'].astype(float)
    df.set_index('opentime')
    return df
def get_all_price(symbols, period='4h'):  
    data = {}
    for s in symbols:
        data[s] = get_price(f'{s.upper()}USDT', period)
        print(f"get price for {s}")
        sleep(0.3)
    return data


symbols = ['BTC','ETH','XRP','LTC','UNI','LINK','EOS','IOST', 'BCH','BNB','TRX']
all = get_all_price(symbols,'1d')

get price for BTC
get price for ETH
get price for XRP
get price for LTC
get price for UNI
get price for LINK
get price for EOS
get price for IOST
get price for BCH
get price for BNB
get price for TRX


In [4]:
from itertools import combinations
from itertools import permutations 
import plotly.express as px
import math

def logreturn_df(all, longsymbols,shortsymbols):
    allsymbols = longsymbols + shortsymbols
    mydict = {}
    for s in allsymbols:
        mydict[s] = all[s].close
    mydict['datetime'] = all[longsymbols[0]].opentime
    df = pd.DataFrame(mydict)
    df.set_index('datetime', inplace=True)
    return df.apply(lambda x: np.log(x) - np.log(x.shift(1))) 
    
def possible(all_shorts):
    return [list(s) for s in combinations(all_shorts, 3)]

def mdd(xs):
    temp = np.maximum.accumulate(xs) - xs
    if 0 in [np.array(temp).size, np.array(xs).size]:
        return -1
    i = np.argmax(temp) # end of the period
    if np.array(xs[:i]).size == 0:
        return -1
    j = np.argmax(xs[:i]) # start of period
    return xs[i] - xs[j]

def key(symbols):
    return ",".join(symbols)

    
def backtest(longsymbols, shortsymbols, log=False, draw=False):
    logreturn = logreturn_df(all, longsymbols, shortsymbols)
    logreturn['long'] = logreturn[longsymbols].mean(axis=1)
    logreturn['short'] = logreturn[shortsymbols].mean(axis=1)
    logreturn['return'] = logreturn['long'] - logreturn['short']
    result = logreturn.filter(['return'], axis=1)
    cumresult = result.cumsum()
    return_tmp = cumresult['return'].dropna().to_numpy()
    return_list = list(return_tmp)
    days = len(return_list)
    daily_return = np.array([math.e**(y-x)-1 for x,y in zip(return_list,return_list[1:])])
    maxdrawdown = mdd(return_tmp)
    if not maxdrawdown:
        return 
    final = cumresult['return'].iat[-1]
    minv = cumresult['return'].min()
    maxv = cumresult['return'].max()
    sharp = np.mean(daily_return)/np.std(daily_return)* (days**0.5)
    k = f"long:[{key(longsymbols)}],short:[{key(shortsymbols)}]"
    if log:
        print(f"{k}, min:{minv} max:{maxv}({100*(math.e**maxv-1)}%), mdd:{maxdrawdown}, sharp:{sharp}")
    if draw:
        fig = px.line(cumresult, x=cumresult.index, y='return')
        fig.show()
#         cumresult.plot(figsize=(15, 10)).axhline(color='black', linewidth=2)
    return k, sharp

def find_combinations(symbols, min_sharp=1.5):
    # 对所有的输入的symbols 进行long/short的排列组合，选择出 sharp ratio 符合条件的组合
    tag = {}
    symbols = [s for s in symbols if all[s] is not None and all[s].shape[0]>300]
    for (longsymbol, shortsymbol) in permutations(symbols,2):
        k, sharp = backtest([longsymbol], [shortsymbol])
        if not math.isnan(sharp) and sharp > min_sharp:
            tag[k] = sharp
    sortedtag = {k: v for k, v in sorted(tag.items(), key=lambda item: item[1], reverse=True)}
    print(sortedtag)
        

backtest(['BTC','ETH'],['XRP','BCH','EOS'], True, True)
# find_combinations(symbols)

long:[BTC,ETH],short:[XRP,BCH,EOS], min:-0.026592099212785356 max:1.6083374268772646(399.4500598776642%), mdd:-0.8286250075366139, sharp:1.9286740689764987


('long:[BTC,ETH],short:[XRP,BCH,EOS]', 1.9286740689764987)